# Evaluating Impact of Dissolved Gases on Total Dissolved Solids (TDS)

Reading the description of the VST syringe filters there is the claim that the filters are "used to quickly remove suspended solids and CO2 gas from ESPRESSO beverages for accurate, repeatable refractometer measurements."[[1][1]]. While the removal of suspended solids seems obvious in having an impact on the refractometer measurements, the impact of CO2 gas is less clear and at the time of writing this am unaware of an experiment (pubically available, hopefully VST has run this experiment) testing this.

This study attempts to look at the impact of dissolved CO2 on TDS measurements.

## Disclosures

* None of the equipment in this experiment was provided to me and represents a setup similar to what I commonly use.
* None of the links included are affiliate links.

## Data Usage

If you intend to use the data for any analysis that is to be shared publicly, provide a way for others to replicate your analysis and reference back to this analysis.

## Caveats

The syringe filters used are NOT the VST syringes as they are quite expensive and it is unclear how they are different from many of the syringe filters you can more cheaply source. A follow up experiment comparing different syringe filters for differences is warranted.


## Equipment

* Coffee:
    * UNK

* Scales:
    * [Acaia Lunar](https://acaia.co/products/lunar) 0.1g readability. Used for measuring espresso weight during pull
    * [Acaia Pearl](https://acaia.co/products/pearl) 0.1g readability. Used to weigh out beans
    * [Ohaus Scout SPX123](https://www.amazon.com/Ohaus-SPX123-Portable-Balance-0-001g/dp/B01AJ089PS/) 0.001g readability. Used for measuring final bean mass in portafilter basket

* Espresso Machine: [Linea Mini](https://home.lamarzoccousa.com/linea-mini/)
* Grinder: [Kafatek Conical Monolith (MC3)](https://www.kafatek.com/index.php/monolith/)
* Tamper: [Decent Tamper V3](https://decentespresso.com/tamper)
* 'Distribution' Tool: [Pullman Chisel](https://pullman.coffee/news/147-chisel-redistribution-tool)
* Basket: [VST 20g Basket](https://store.vstapps.com/products/vst-precision-filter-baskets)
* Water: Distill 1 Gallon and with one packet of [Third Wave Water Espresso Profile](https://thirdwavewater.com/products/1-gallon-espresso-profile?variant=32477449289774) added
* [Metal Shot Glasses for Cooling](https://www.amazon.com/gp/product/B07SX6WG6M/)
  * Chose metal in an attempt to reduce the time taken for samples to reach room temp
* [Laptop Cooling Pad](https://www.amazon.com/gp/product/B07K2QPYMY/)
* [0.45μm syringe filters](https://www.amazon.com/gp/product/B07X2NBRFD/)
    * A much cheaper syringe filter than the VST filters

## Espresso Methodology

### Pre-experiment Preparation

1. Calibrate Acaia Lunar/Pearl using method described [here](https://help.acaia.co/hc/en-us/articles/360035746811-How-to-calibrate-your-Lunar-Pearl-Pearl-Model-S-Pearl-2021-or-Pyxis-scale)
2. Calibrate Ohaus Scout using method described in manual [here](https://us.ohaus.com/en-us/products/balances-scales/portable-balances/scout-spx/spx123-am)

### Pulling Shot

1. Tare Ohaus scale with portafilter basket
1. Weigh out 18.1g of coffee beans using the Acaia Lunar
1. Grind beans into Portafilter, pouring beans into the grinder all at once, use billows to push out grounds
1. Tap portafilter on workspace twice, flatten bed using 'distribution' tool, then tamp
1. Weight final weight of beans
1. Place portafilter in machine, place cup on scale, tare.
1. Engage machine and scale timer at same time
1. Disengage machine when weight in cup reaches 36.0g
1. Record final weight, pull time of shot as well as date and time
1. Proceed to [measure TDS](#Measuring-TDS) using the three different TDS measurements

## Measuring TDS

There are three methods with which TDS will be measured for this experiment, they are as follows:

1. Unfiltered espresso
2. Unfiltered espresso with gases removed
3. Filtered espresso


### Collect Samples

1. Stir shot with a [Spoon](#Stirring-With-a-Spoon)
1. Use 3ml syringe to [extract](#Extracting-Sample-Procedure) two 3ml samples, from below crema, place into metal 'shot glass' to [cool](#Cooling-Procedure).
1. Wait 5 minutes for cooling
1. For each method, use 1.0ml samples, perform the following steps
    1. Clean refractometer sensor, tare using distilled water
    1. Draw up 1.0ml of liquid in new syringe
    1. If degassing, perform [degassing procedure](#Degassing-Procedure)
    1. If filtering, attach new filter
    1. Dispose of first 4 drops
    1. Fill refractometer sensor area
    1. Collect TDS and Temperature (from refractometer) five times, once every 10 seconds  


###  Stirring With a Spoon

1. Stir shot with a small spoon ten times clockwise and ten times counterclockwise

### Degassing Procedure

1. Place finger over the opening of the syringe
1. Pull back on the plunger to create a vacuum and pull gas out of sample, repeat until no more air bubbles form


## Cooling Procedure

1. Use metal shot glasses to cool samples
1. Place samples on laptop cooling pad, turn on full


### Extracting Sample Procedure

When extracting a sample using a syringe, place the syringe underneath the crema and draw into syringe and push back into the cup three times. Keep the fourth draw.[[2][2]]


## References

* \[1\] [Syringe Filters - VST Inc][1]
* \[2\] [Centrifuging Espresso - A Waste of Coffee][2]


[1]: <https://store.vstapps.com/products/syringe-filters-for-espresso-coffee> "Syringe Filters - VST Inc"
[2]: <https://awasteof.coffee/2019/02/12/centrifuging-espresso/> "Centrifuging Espresso - A Waste of Coffee"

In [2]:
import os
import math

from collections import defaultdict
from tempfile import NamedTemporaryFile

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from scipy.stats import (
    kendalltau,
    pearsonr,
    ks_2samp,
    shapiro,
    spearmanr,
    ttest_ind,
    bootstrap,
    theilslopes,
    linregress,
)

from IPython.core.display import display

# Generate ordering of samples

In [6]:
# Seed the RNG to ensure consistent results across notebook runs
np.random.seed(814)

samples = np.asarray(["unfiltered", "unfiltered degassed", "filtered"])
number_of_pairs = 10 # Run an equal number of pairs
count = 0
for _ in range(number_of_pairs):
    np.random.shuffle(samples)
    count += 1
    print(f"Sample {count} testing order")
    for val in samples:
        print(val) 
    print("-----")

Sample 1 testing order
unfiltered
filtered
unfiltered degassed
-----
Sample 2 testing order
unfiltered
filtered
unfiltered degassed
-----
Sample 3 testing order
unfiltered
filtered
unfiltered degassed
-----
Sample 4 testing order
unfiltered degassed
filtered
unfiltered
-----
Sample 5 testing order
unfiltered degassed
filtered
unfiltered
-----
Sample 6 testing order
filtered
unfiltered
unfiltered degassed
-----
Sample 7 testing order
filtered
unfiltered
unfiltered degassed
-----
Sample 8 testing order
filtered
unfiltered
unfiltered degassed
-----
Sample 9 testing order
unfiltered degassed
filtered
unfiltered
-----
Sample 10 testing order
filtered
unfiltered
unfiltered degassed
-----


In [3]:
DATA_DOWNLOAD_URL = "https://docs.google.com/spreadsheets/d/1dDfuIq74pjELNtfb_0sdiNnG3qFKrfqC2sax7ewrWYg/export?format=csv&gid=1739174083"
plt.rcParams["figure.dpi"] = 150
plt.rcParams["axes.axisbelow"] = True

In [4]:
# Download CSV, won't work on windozes
with NamedTemporaryFile(suffix=".csv") as temp:
    with open(temp.name, "wb") as ofs:
        resp = requests.get(DATA_DOWNLOAD_URL, stream=True)
        for chunk in resp.iter_content():
            ofs.write(chunk)
    df = pd.read_csv(temp.name)
# Clean up the CSV file.
for char in [' ', '(', ')', "%", "/", "\\", '"']:
    df.columns = df.columns.str.strip().str.lower().str.replace(char, '')
df = df[pd.to_numeric(df["grindersetting"], errors="coerce").notnull()]
# Only look at data with TDS values and ratings
df = df.assign(extraction_yield = (df.tds * df.output) / df.coffeegrams)
df.date = pd.to_datetime(df.date, infer_datetime_format=True, utc=True)
df.roastdate = pd.to_datetime(df.roastdate, infer_datetime_format=True, utc=True)
df = df.assign(days_off_roast = df.date - df.roastdate)

evaporated_df = df[~df["containerweight+driedsolids"].isna()]
evaporated_df = evaporated_df.assign(
    tds=np.abs(
        (evaporated_df["containerweight+driedsolids"] - evaporated_df["containerweight"]) / evaporated_df.liquidweight * 100
    )
)
evaporated_df = evaporated_df.assign(extraction_yield = (evaporated_df.tds * evaporated_df.output) / evaporated_df.coffeegrams)
df.drop(df[df.tds.isna()].index, inplace=True)

# Combine the two DataFrames
ts_val = np.zeros_like(df.tds.values)
extract_yield = np.zeros_like(ts_val)
for i, d in enumerate(df["sample"].values):
    ts_val[i] = evaporated_df[evaporated_df["sample"] == d].tds.values[0]
    extract_yield[i] = evaporated_df[evaporated_df["sample"] == d].extraction_yield.values[0]
df = df.assign(
    total_solids=ts_val,
    total_solids_extraction=extract_yield,
)

In [7]:
def bootstrap_statistic(x, y, func, samples=1000):
    """Naive Bootstrap of a paired function"""
    sample_vals = []
    x = np.asarray(x)
    y = np.asarray(y)
    for _ in range(samples):
        idxs = np.random.randint(0, len(x), len(x))
        sample_vals.append(func(x[idxs], y[idxs]))
    return sample_vals

## Hypothesis #1: Dissolved C02 Gases have no impact 

Dissolved CO2 gases will have no impact on the TDS measurements.